In [22]:
import http.client
import json
import ssl
import certifi
import pandas as pd
from datetime import datetime, timedelta

# Establish the HTTPS connection
conn = http.client.HTTPSConnection("linkedin-data-api.p.rapidapi.com", context=ssl.create_default_context(cafile=certifi.where()))

# Set the headers
headers = {
    'x-rapidapi-key': "fac8018a9dmshb2ea5c999eb0475p188109jsnf410f35e4ed9",
    'x-rapidapi-host': "linkedin-data-api.p.rapidapi.com"
}

# Calculate the date 7 days ago
seven_days_ago = datetime.now() - timedelta(days=7)

# Format the date in YYYY-MM-DD for the query
seven_days_ago_str = seven_days_ago.strftime('%Y-%m-%d')

# Example query to search for Data Scientist jobs listed in the last 7 days
query = f"/search-jobs?title=Data+Scientist"

# Make the request to search jobs
conn.request("GET", query, headers=headers)

# Get the response and read the data
res = conn.getresponse()
search_data = res.read()

# Decode the JSON response from the search
search_json_data = json.loads(search_data.decode("utf-8"))

# Extract job details
jobs_list = search_json_data.get('data', [])

# Initialize an empty DataFrame to store all job details
all_jobs_df = pd.DataFrame()

# Process each job in the list
for job_data in jobs_list:
    # Get the title, workplace, and listed date
    title = job_data.get('title', '')
    work_place = job_data.get('workPlace', '')
    listed_at_date_str = job_data.get('listedAtDate', '')

    # Convert listedAtDate to datetime
    if listed_at_date_str:
        listed_at_date = datetime.strptime(listed_at_date_str.split(" +")[0], '%Y-%m-%d %H:%M:%S')
        
        # Check conditions: title is "Data Scientist", workplace is "Remote", and listed within the last 7 days
        if title.lower() == 'data scientist' and work_place.lower() == 'remote' and listed_at_date >= seven_days_ago:
            # Normalize to DataFrame and append
            df = pd.json_normalize(job_data)
            all_jobs_df = pd.concat([all_jobs_df, df], ignore_index=True)

job_data

[]

In [23]:
print(search_json_data)


{'message': 'You have exceeded the MONTHLY quota for Credits on your current plan, BASIC. Upgrade your plan at https://rapidapi.com/rockapis-rockapis-default/api/linkedin-data-api'}
